<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Sntence_entailment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**work in progress**

# Data Exploration

In [1]:
import numpy as np
import pandas as pd

In [6]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
!unzip train.zip

In [ ]:
train = pd.read_csv("train.csv")

In [ ]:
train_filtered  = train.filter(['premise','hypothesis','label'])

In [ ]:
train_filtered.head()

,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1


In [ ]:
#get cell value
train.iloc[1]['premise']
train.iloc[1]['hypothesis']

In [ ]:
dataset = nlp.Dataset.from_pandas(train_filtered)
ds_splitted = dataset.train_test_split(test_size=0.1)

In [ ]:
ds_splitted['train']['premise'][1]

"because i always had to do it and so i just pay someone else to do it and they do the they do the cutting they fertilize they um edge and um i think this year i'm going to have some landscaping put in"

# Model

In [2]:
%%capture
!pip install pytorch_lightning
!pip install transformers
!pip install nlp

In [3]:
import torch as th
import pytorch_lightning as pl
import nlp
import transformers

In [4]:
from transformers import BertTokenizer

In [ ]:
#example of tokenizing 2 sentences into
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
sequence_a = ds_splitted['train']['premise'][1]
sequence_b = ds_splitted['train']['hypothesis'][1]
encoded_dict = tokenizer(sequence_a, sequence_b,                     
                         max_length=128, 
                         truncation = True,
                         pad_to_max_length=True,
                         return_tensors = 'pt')
#decoded = tokenizer.decode(encoded_dict["input_ids"])
print(type(encoded_dict))

In [ ]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')
outputs = model(**encoded_dict)

In [ ]:
outputs[1].shape

torch.Size([1, 768])

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs[0]

In [13]:
m = th.nn.ReLU()
type(m)

torch.nn.modules.activation.ReLU

In [43]:
class Sentences(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.brt = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.layer1 = th.nn.Linear(768,32)
        self.relu = th.nn.ReLU()
        self.layer2 = th.nn.Linear(32,3)
        self.loss = th.nn.CrossEntropyLoss(reduction='none')

    def prepare_data(self):
        tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

        def _tokenize(x):
            print('in tokenize')
            return tokenizer(
                    x['premise'],
                    x['hypothesis'],
                    max_length=64, 
                    truncation = True,
                    pad_to_max_length=True)
            
        def _prepare_ds():
            df = pd.read_csv('train.csv', nrows=50)
            df_filtered  = df.filter(['premise','hypothesis','label'])
            dataset = nlp.Dataset.from_pandas(df_filtered)
            ds_flt = dataset.train_test_split(test_size=0.1)

            ds_flt['train'] = ds_flt['train'].map(_tokenize, batched=True)
            ds_flt['train'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])

            ds_flt['test'] = ds_flt['test'].map(_tokenize, batched=True)
            ds_flt['test'].set_format(type='torch',columns = ['input_ids','token_type_ids','label','attention_mask'])

            return ds_flt['train'], ds_flt['test']

        self.train_ds, self.test_ds = _prepare_ds()      

    def forward(self, input_ids, masks, token_type_ids):
        logits = self.brt(input_ids, masks, token_type_ids)
        print(logits[0].shape)
        print(logits[1].shape)
        out = self.layer1(logits[0])
        print('layer1 output - {}'.format(out.shape))
        out = self.relu(out)
        print('RELU output - {}'.format(out.shape))
        out = self.layer2(out)
        print('layer2 output - {}'.format(out.shape))
        return out

    def training_step(self, batch, batch_idx):
        #print('training step function')
        logits = self.forward(batch['input_ids'],batch['attention_mask'], batch['token_type_ids'])
        #print('in  training_step - logits'.format(logits.shape))
        #print('in  training_step - labels'.format(batch['labels'].shape))
        loss = self.loss(logits, batch['label']).mean()

        return {'loss': loss, 'log': {'train_loss': loss}}

    def validation_step(self, batch, batch_idx):
        print('validation step function')
        logits = self.forward(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
        print('in  validation_step logits shape- {}'.format(logits.shape))
        print('in  validation_step label shape - {}'.format(batch['label'].shape))
        print('in  validation_step labels- {}'.format(batch['label'].numpy()))
        loss = self.loss(logits, batch['label'])
        acc = (logits.argmax(-1) == batch['label']).float()

        return {'loss': loss, 'acc': acc}

    def validation_epoch_end(self, outputs):
        loss = th.cat([o['loss'] for o in outputs], 0).mean()
        acc = th.cat([o['acc'] for o in outputs], 0).mean()
        out = {'val_loss': loss, 'val_acc': acc}
        return {**out, 'log': out}

    def train_dataloader(self):
        #print('train_dataloader')
        return th.utils.data.DataLoader(
                self.train_ds,
                batch_size=1,
                drop_last=True,
                shuffle=True,
                )

    def val_dataloader(self):
        #print('val_dataloader')
        return th.utils.data.DataLoader(
                self.test_ds,
                batch_size=1,
                drop_last=False,
                shuffle=True,
                )

    def configure_optimizers(self):
        return th.optim.SGD(
            self.parameters(),
            lr=0.01,
            momentum=0.9,
        )

In [41]:

def model_fit():
    model = Sentences()
    trainer = pl.Trainer(
        #default_root_dir='logs',
        gpus=(1 if th.cuda.is_available() else 0),
        max_epochs=4,
        fast_dev_run=False,
        logger=pl.loggers.TensorBoardLogger('logs/', name='imdb'),
    )
    trainer.fit(model)

!rm -rf ./logs/

model_fit()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores



in tokenize


in tokenize

in tokenize


in tokenize



/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name   | Type             | Params
--------------------------------------------
0 | brt    | BertModel        | 109 M 
1 | layer1 | Linear           | 24 K  
2 | relu   | ReLU             | 0     
3 | layer2 | Linear           | 99    
4 | loss   | CrossEntropyLoss | 0     
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


validation step function
torch.Size([1, 64, 768])
torch.Size([1, 768])
layer1 output - torch.Size([1, 64, 32])
RELU output - torch.Size([1, 64, 32])
layer2 output - torch.Size([1, 64, 3])
in  validation_step logits shape- [[[-0.01061276  0.09521037  0.21028438]
  [ 0.04038514  0.06856761  0.11722112]
  [ 0.26182336  0.2398583  -0.05763215]
  [ 0.26005796  0.17213774  0.08049655]
  [ 0.26752424  0.12409272  0.0472457 ]
  [ 0.259698    0.24964502  0.06779353]
  [ 0.09331541  0.2025837   0.15845767]
  [ 0.1283965   0.2654156   0.16701387]
  [ 0.1895185   0.13126901  0.18734798]
  [ 0.2583549   0.05492248  0.12007427]
  [ 0.15154606  0.24827811  0.03821656]
  [ 0.22703686  0.12942156  0.16794014]
  [ 0.25520706  0.21277621  0.16732392]
  [ 0.20974079  0.24204579  0.14546922]
  [ 0.19599108  0.35013843  0.17190026]
  [ 0.11525911  0.12018948  0.15931949]
  [ 0.31119695  0.11798627  0.23258969]
  [ 0.22781062  0.29531658  0.07799413]
  [ 0.25618923  0.06623289  0.12617293]
  [ 0.2577057   0.

ValueError: ignored

# Sandbox